In [ ]:
import numpy as np
import pydot
from IPython.display import HTML, SVG, display
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    GenerateHtml,
    InverseDynamicsController,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    Simulator,
    StartMeshcat,
    RigidTransform,
    RollPitchYaw,
    PiecewisePolynomial,
    TrajectorySource,
    Solve,
    RotationMatrix,
    eq
)
from manipulation.meshcat_utils import AddMeshcatTriad
from pydrake.multibody import inverse_kinematics
from manipulation import running_as_notebook

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://7e656c9a-e35c-4740-b249-07fef330708d.deepnoteproject.com/7004/


# Example with just iiwa on rail

In [ ]:
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
# Note that we parse into both the plant and the scene_graph here.
iiwa_model = Parser(plant, scene_graph).AddModelsFromUrl(
    "file:///work/iiwa7/iiwa7_on_rail.sdf"
)[0]

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rail"))
plant.Finalize()

# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Adds an approximation of the iiwa controller.
# TODO(russt): replace this with the joint impedance controller.
kp = [100] * plant.num_positions()
ki = [1] * plant.num_positions()
kd = [20] * plant.num_positions()
iiwa_controller = builder.AddSystem(
    InverseDynamicsController(plant, kp, ki, kd, False)
)
iiwa_controller.set_name("iiwa_controller")
builder.Connect(
    plant.get_state_output_port(iiwa_model),
    iiwa_controller.get_input_port_estimated_state(),
)
builder.Connect(
    iiwa_controller.get_output_port_control(), plant.get_actuation_input_port()
)
diagram = builder.Build()
diagram.set_name("with iiwa controller")

context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = np.array([0.3,-1.57, 0.1, 0, -1.2, 0, 1.6, 0])
x0 = np.hstack((q0, 0 * q0))
plant.SetPositions(plant_context, q0)
iiwa_controller.GetInputPort("desired_state").FixValue(
    iiwa_controller.GetMyMutableContextFromRoot(context), x0
)

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0 if running_as_notebook else 0.1);

# With path following on sinusoid - bed, rail, robot

In [ ]:
def BuildAndSimulateTrajectory(q_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    """
    builder = DiagramBuilder()

    # Adds both MultibodyPlant and the SceneGraph, and wires them together.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    # Note that we parse into both the plant and the scene_graph here.
    iiwa_model = Parser(plant, scene_graph).AddModelsFromUrl(
    "file:///work/iiwa7/iiwa7_on_rail.sdf"
    )[0]


    print_bed = Parser(plant, scene_graph).AddModels("file:///work/cylinder.sdf")[0]

    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rail"))

    bed_transform = RigidTransform()
    bed_transform.set_rotation(RollPitchYaw(0,np.pi/2,0))
    bed_transform.set_translation(np.array([1.5,-1,1]))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rotator_Base"), bed_transform)
    plant.Finalize()

    # Adds the MeshcatVisualizer and wires it to the SceneGraph.
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    # Adds an approximation of the iiwa controller.
    # TODO(russt): replace this with the joint impedance controller.
    kp = [100] * plant.num_positions()
    ki = [1] * plant.num_positions()
    kd = [20] * plant.num_positions()
    iiwa_controller = builder.AddSystem(
        InverseDynamicsController(plant, kp, ki, kd, False)
    )
    iiwa_controller.set_name("iiwa_controller")
    builder.Connect(
        plant.get_state_output_port(),
        iiwa_controller.get_input_port_estimated_state(),
    )
    builder.Connect(
        iiwa_controller.get_output_port_control(), plant.get_actuation_input_port()
    )


    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))


    builder.Connect(
        q_traj_system.get_output_port(), iiwa_controller.GetInputPort("desired_state")
    )


    diagram = builder.Build()
    diagram.set_name("with iiwa controller")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = np.array([0, -1.57, 0.1, 0, -1.2, 0, 1.6, 0, 0])
    x0 = np.hstack((q0, 0 * q0))
    plant.SetPositions(plant_context, q0)


    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)
    visualizer.StartRecording(False)
    simulator.AdvanceTo(5.0 if running_as_notebook else 0.1);
    visualizer.PublishRecording()

    return simulator, plant

In [ ]:
def create_q_knots_sinusoid(t_lst):
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    
    q_nominal = np.array(
        [0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0, 0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0]
    )  # nominal joint angles for joint-centering.
    


    for i in range(len(t_lst)):
        this_q = np.sin(t_lst[i])*np.ones(18)
        q_knots.append(this_q)

    return q_knots

In [ ]:
t_lst = np.linspace(0, 11, 30)

q_knots = np.array(create_q_knots_sinusoid(t_lst))
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:18].T)

print(q_traj)

simulator, station_plant = BuildAndSimulateTrajectory(q_traj, 11.0)

# Using real path

In [ ]:
def BuildAndSimulateTrajectory(q_traj, duration=0.01):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    """
    builder = DiagramBuilder()

    # Adds both MultibodyPlant and the SceneGraph, and wires them together.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    # Note that we parse into both the plant and the scene_graph here.
    iiwa_model = Parser(plant, scene_graph).AddModelsFromUrl(
    "file:///work/iiwa7/iiwa7_on_rail.sdf"
    )[0]


    print_bed = Parser(plant, scene_graph).AddModels("file:///work/cylinder.sdf")[0]

    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rail"))

    bed_transform = RigidTransform()
    bed_transform.set_rotation(RollPitchYaw(0,np.pi/2,0))
    bed_transform.set_translation(np.array([1.5,-1,1]))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rotator_Base"), bed_transform)
    plant.Finalize()

    # Adds the MeshcatVisualizer and wires it to the SceneGraph.
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    # Adds an approximation of the iiwa controller.
    # TODO(russt): replace this with the joint impedance controller.
    kp = [100] * plant.num_positions()
    ki = [1] * plant.num_positions()
    kd = [20] * plant.num_positions()
    iiwa_controller = builder.AddSystem(
        InverseDynamicsController(plant, kp, ki, kd, False)
    )
    iiwa_controller.set_name("iiwa_controller")
    builder.Connect(
        plant.get_state_output_port(),
        iiwa_controller.get_input_port_estimated_state(),
    )
    builder.Connect(
        iiwa_controller.get_output_port_control(), plant.get_actuation_input_port()
    )


    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))


    builder.Connect(
        q_traj_system.get_output_port(), iiwa_controller.GetInputPort("desired_state")
    )


    diagram = builder.Build()
    diagram.set_name("with iiwa controller")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = np.array([ 0.6,         -0.24161793,  0.1128608,   0.0,         -1.2628608,   0.7981223,
   0.37067477,  0.7197221,   0.0        ])
    x0 = np.hstack((q0, 0 * q0))
    plant.SetPositions(plant_context, q0)


    AddMeshcatTriad(meshcat, "bed", length=0.15, radius=0.006, X_PT=bed_transform)


    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)
    visualizer.StartRecording(False)

    SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg())

    for i in range(0,6):
        simulator.AdvanceTo(i);

        # Get the context after initialization
        plant_context = plant.GetMyContextFromRoot(context)

        # Get the rigid transform of the frame in the world frame
        transform = plant.CalcRelativeTransform(plant_context, plant.world_frame(), plant.GetFrameByName("iiwa_link_7"))

        # Print the resulting transform
        #print(f"Rigid Transform of frame '{frame_name}' in the world frame:")
        #print(transform)
        #AddMeshcatTriad(meshcat, "bed" + str(i), length=0.15, radius=0.006, X_PT=transform)


    visualizer.PublishRecording()

    return simulator, plant

In [ ]:
def CreateIiwaControllerPlant():
    """creates plant that includes only the robot and gripper, used for controllers."""
    builder = DiagramBuilder()

    # Adds both MultibodyPlant and the SceneGraph, and wires them together.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    # Note that we parse into both the plant and the scene_graph here.
    iiwa_model = Parser(plant, scene_graph).AddModelsFromUrl(
    "file:///work/iiwa7/iiwa7_on_rail.sdf"
    )[0]


    print_bed = Parser(plant, scene_graph).AddModels("file:///work/cylinder.sdf")[0]

    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rail"))

    bed_transform = RigidTransform()
    bed_transform.set_rotation(RollPitchYaw(0,np.pi/2,0))
    bed_transform.set_translation(np.array([1.5,-1,1]))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rotator_Base"), bed_transform)
    plant.Finalize()


    diagram = builder.Build()
    diagram.set_name("with iiwa controller")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = np.array([0, -1.57, 0.1, 0, -1.2, 0, 1.6, 0, 0])
    x0 = np.hstack((q0, 0 * q0))
    plant.SetPositions(plant_context, q0)

    return plant

In [ ]:
def create_q_knots(t_lst):
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    plant = CreateIiwaControllerPlant()
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("iiwa_link_7")
    q_nominal = np.array(
        [ 0.6,         -0.24161793,  0.1128608,   0.0,         -1.2628608,   0.7981223,
   0.37067477,  0.7197221,   0.0        ]
    )  # nominal joint angles for joint-centering.

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=R_WG,
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=bounds,
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame,
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower,
            p_AQ_upper=p_WG_upper,
        )

    for i in range(len(t_lst)):
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q()  # Get variables for MathematicalProgram
        prog = ik.prog()  # Get MathematicalProgram

        #### Modify here ###############################

        ################################################


        #AddPositionConstraint(ik, pose_lst[i].translation() - np.array([0.,0.,0.01]), pose_lst[i].translation()+np.array([0.,0.,0.01]))
        #AddOrientationConstraint(ik, pose_lst[i].rotation(), 0.1)
        q = ik.q()
        prog.AddQuadraticErrorCost(np.identity(len(q)), q_nominal, q)

        #locking the extra joint
        prog.AddConstraint(q[3]== 0)

        des_rot = RigidTransform()
        des_rot.set_rotation(RollPitchYaw(np.pi/2,0,np.pi/2))
        AddOrientationConstraint(ik, des_rot.rotation(), 0.01)
        des_pos = np.array([0,0.8,-0.2])

        ik.AddPositionConstraint(
            frameA=plant.GetFrameByName("Build_Plate"),
            frameB=gripper_frame,
            p_BQ=np.zeros(3),
            p_AQ_lower=des_pos-np.array([0.01,0.01,0.01]),
            p_AQ_upper=des_pos+np.array([0.01,0.01,0.01]),
        )

        prog.AddConstraint(q[8]== -0.3+np.sin(t_lst[i])*0.3)
        prog.AddConstraint(q[0]==0.8)
        #prog.AddConstraint(q[2]==0)
        

        if i == 0:
            prog.SetInitialGuess(q, q_nominal)
        else:
            prog.SetInitialGuess(q, last_solution)
        result = Solve(prog)

        assert result.is_success()
        last_solution = result.GetSolution(q_variables)
        q_knots.append(last_solution)

    return q_knots

In [ ]:
"""Simulate trajectory for manipulation station.
@param q_traj: Trajectory class used to initialize TrajectorySource for joints.
"""
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
# Note that we parse into both the plant and the scene_graph here.
iiwa_model = Parser(plant, scene_graph).AddModelsFromUrl(
"file:///work/iiwa7/iiwa7_on_rail.sdf"
)[0]


print_bed = Parser(plant, scene_graph).AddModels("file:///work/cylinder.sdf")[0]

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rail"))

bed_transform = RigidTransform()
bed_transform.set_rotation(RollPitchYaw(0,np.pi/2,0))
bed_transform.set_translation(np.array([1.5,-1,1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("Rotator_Base"), bed_transform)
plant.Finalize()

# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Adds an approximation of the iiwa controller.
# TODO(russt): replace this with the joint impedance controller.
kp = [100] * plant.num_positions()
ki = [1] * plant.num_positions()
kd = [20] * plant.num_positions()
iiwa_controller = builder.AddSystem(
    InverseDynamicsController(plant, kp, ki, kd, False)
)
iiwa_controller.set_name("iiwa_controller")
builder.Connect(
    plant.get_state_output_port(),
    iiwa_controller.get_input_port_estimated_state(),
)
builder.Connect(
    iiwa_controller.get_output_port_control(), plant.get_actuation_input_port()
)


q_traj_system = builder.AddSystem(TrajectorySource(q_traj))


builder.Connect(
    q_traj_system.get_output_port(), iiwa_controller.GetInputPort("desired_state")
)


diagram = builder.Build()
diagram.set_name("with iiwa controller")

context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = np.array([ 0.6,         -0.24161793,  0.1128608,   0.0,         -1.2628608,   0.7981223,
0.37067477,  0.7197221,   0.0        ])
x0 = np.hstack((q0, 0 * q0))
plant.SetPositions(plant_context, q0)


AddMeshcatTriad(meshcat, "bed", length=0.15, radius=0.006, X_PT=bed_transform)


simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
visualizer.StartRecording(False)


# Get the context after initialization
plant_context = plant.GetMyContextFromRoot(context)

# Get the rigid transform of the frame in the world frame
#transform = plant.CalcRelativeTransform(plant_context, plant.world_frame(), plant.GetFrameByName("iiwa_link_7"))


In [ ]:
t_lst = np.linspace(0, 11, 30)
q_knots = np.array(create_q_knots(t_lst))


double_q_knots = np.hstack((q_knots, q_knots*0))

q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, double_q_knots[:, 0:18].T)


simulator, station_plant = BuildAndSimulateTrajectory(q_traj, 11.0)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7e656c9a-e35c-4740-b249-07fef330708d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>